# Acquiring and Processing Information on the World's Largest Banks

In this project, we will work with real-world data and perform the operations of Extraction, Transformation, and Loading (ETL) as required.

- Task 1: Logging function
- Task 2: Extraction of data
- Task 3: Transformation of data
- Task 4: Loading to CSV
- Task 5: Loading to Database
- Task 6: Function to Run queries on Database
- Task 7: Verify log entries

## Preliminaries: Installing libraries and downloading data

Install the required libraries

In [17]:
%pip install numpy pandas bs4 wget requests

     ---------------------------------------- 64.8/64.8 kB 1.2 MB/s eta 0:00:00
     -------------------------------------- 162.7/162.7 kB 3.2 MB/s eta 0:00:00
     -------------------------------------- 129.8/129.8 kB 7.5 MB/s eta 0:00:00
     -------------------------------------- 105.8/105.8 kB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 70.4/70.4 kB 3.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Download the required exchange rate file

In [7]:
# Download the CSV data first into a local `exchange_rate.csv` file
import wget
wget.download("https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks")

'List_of_largest_banks'

Importing the required libraries:

In [18]:
import requests
import sqlite3
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# Suppress generated warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

## Tasks

### Task 1: Logging function

This function accepts the message to be logged and enters it to the log file on a new line.

Log entries will be in the format: `<timestamp> : <message>`

In [20]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''

    time_stamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"{time_stamp} : {message}\n"
    with open("code_log.txt", "a") as log_file:
        log_file.write(log_entry)

### Task 2: Extraction of data

1. Inspect the URL and identify the position and pattern of the tabular information in the HTML code under the heading `By market capitalization`. In the given webpage, our table is at the first position, or index 0 and we require the entries under *"Bank name"* and *"Market cap (US$ billion)"*.

    <img src="List_of_largest_banks.png">

    Notice that all rows under this table are mentioned as `tr` objects under the table. Clicking one of them reveals that the data in each row is further saved as a `td` object. 
    
    It is also important to note that entries under "Bank name" have two hyperlinks associated with it, one for the Country and the other for Bank name, as seen in the image above.

2. Write the function `extract()` to retrieve the information of the table to a Pandas DataFrame.

In [19]:
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''

    page = requests.get(url).text
    soup = BeautifulSoup(page, "html.parser")

    df = pd.DataFrame(columns=table_attribs)

    tables = soup.find_all("tbody")
    rows = tables[0].find_all("tr")

    for row in rows:
        col = row.find_all("td")
        if len(col) != 0:
            data_dict = {"Name": col[1].find_all("a")[1]["title"],
                         "MC_USD_Billion": float(col[2].contents[0][:-1])}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df, df1], ignore_index=True)

    return df

The code functions as follows:

1. Extract the web page as text.
1. Parse the text into an HTML object.
1. Create an empty Pandas DataFrame named `df` with columns argument set as `table_attribs`.
1. Extract all `tbody` attributes of the HTML object and then extract all the rows of the index 0 table using the `tr` attribute.
1. Iterate over the contents of the variable `rows`.
1. Extract all the `td` data objects in the row and save them to `col`.
1. Check if the length of `col` is 0, that is, if there is no data in the current row. This is important since, many times there are merged rows that are not apparent in the web page appearance.
1. Create a dictionary `data_dict` with the keys same as the columns of the dataframe created for recording the output earlier and corresponding values from the second and third headers of data.
    - Extract the `title` attribute of the second hyperlink from the `Bank name` column.
    - Remove the last character (`\n`) from the `Market cap (US$ billion)` column contents using negative index slicing, then typecast the value to `float` format.
1. Convert the dictionary to a dataframe and concatenate it with the existing one. This way, the data keeps getting appended to the dataframe with every iteration of the loop.
1. Return the dataframe.

### Task 3: Transformation of data

The Transform function needs to perform the following tasks:

1. Read the exchange rate CSV file and convert the contents to a dictionary so that the contents of the first columns are the keys to the dictionary and the contents of the second column are the corresponding values.

1. Add 3 different columns to the dataframe: `MC_GBP_Billion`, `MC_EUR_Billion` and `MC_INR_Billion`, each containing the content of MC_USD_Billion scaled by the corresponding exchange rate factor. Round the resulting data to 2 decimal places.

In [21]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

    # Read exchange rate CSV file
    exchange_rate = pd.read_csv(csv_path)

    # Convert to a dictionary with "Currency" as keys and "Rate" as values
    exchange_rate = exchange_rate.set_index("Currency").to_dict()["Rate"]

    # Add MC_GBP_Billion, MC_EUR_Billion, and MC_INR_Billion
    # columns to dataframe. Round off to two decimals
    df["MC_GBP_Billion"] = [np.round(x * exchange_rate["GBP"], 2) for x in df["MC_USD_Billion"]]
    df["MC_EUR_Billion"] = [np.round(x * exchange_rate["EUR"], 2) for x in df["MC_USD_Billion"]]
    df["MC_INR_Billion"] = [np.round(x * exchange_rate["INR"], 2) for x in df["MC_USD_Billion"]]
    
    return df

### Task 4: Loading to CSV

Load the transformed dataframe to an output CSV file.

In [22]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

    df.to_csv(output_path)

### Task 5: Loading to Database

Load the transformed dataframe to a SQL database server as a table.

In [23]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''

    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

### Task 6: Function to Run queries on Database

Run queries on the database table.

In [24]:
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''

    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

## Running ETL Process

Declaring known values:

In [25]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
csv_path = "./exchange_rate.csv"
table_attribs = ["Name", "MC_USD_Billion"]
output_path = "./Largest_banks_data.csv"
db_name = "Banks.db"
table_name = "Largest_banks"
log_file = "./code_log.txt"

log_progress("Preliminaries complete. Initiating ETL process")

Call **extract()** function:

In [26]:
df = extract(url, table_attribs)
print(df)

log_progress("Data extraction complete. Initiating Transformation process")

                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                                     HSBC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81


Call **transform()** function:

In [27]:
df = transform(df, csv_path)
print(df)

log_progress("Data transformation complete. Initiating Loading process")

                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

Call **load_to_csv()** function:

In [28]:
load_to_csv(df, output_path)

log_progress("Data saved to CSV file")

Initiate **SQLite3** connection:

In [29]:
sql_connection = sqlite3.connect(db_name)

log_progress("SQL Connection initiated")

Call **load_to_db()** function:

In [30]:
load_to_db(df, sql_connection, table_name)

log_progress("Data loaded to Database as a table, Executing queries")

Call **run_query()** function: 

1. Print the contents of the entire table

In [31]:
query_statement = f"SELECT * from {table_name}"
run_query(query_statement, sql_connection)

SELECT * from Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2    

2. Print the average market capitalization of all the banks in Billion GBP

In [32]:
query_statement = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
run_query(query_statement, sql_connection)

SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0              151.987


3. Print only the names of the top 5 banks

In [33]:
query_statement = f"SELECT Name from {table_name} LIMIT 5"
run_query(query_statement, sql_connection)

log_progress("Process Complete")

SELECT Name from Largest_banks LIMIT 5
                                      Name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank


Close **SQLite3** connection:

In [34]:
sql_connection.close()

log_progress("Server Connection closed")

#### Task 7: Verify log entries

Upon successful completion of execution, you should see all the relevant entries made in the log file in relation to the stages of code execution.

In [35]:
with open(log_file, "r") as log:
    LogContent = log.read()
    print(LogContent)

2025-07-15 11:08:18 : Preliminaries complete. Initiating ETL process
2025-07-15 11:08:18 : Data extraction complete. Initiating Transformation process
2025-07-15 11:08:18 : Data transformation complete. Initiating Transformation process
2025-07-15 11:08:18 : Data saved to CSV file
2025-07-15 11:08:18 : SQL Connection initiated
2025-07-15 11:08:18 : Data loaded to Database as a table, Executing queries
2025-07-15 11:08:18 : Process Complete
2025-07-15 11:08:18 : Server Connection closed
2025-07-17 00:09:06 : Preliminaries complete. Initiating ETL process
2025-07-17 00:09:26 : Data extraction complete. Initiating Transformation process
2025-07-17 00:09:42 : Data transformation complete. Initiating Loading process
2025-07-17 00:09:54 : Data saved to CSV file
2025-07-17 00:10:09 : SQL Connection initiated
2025-07-17 00:10:21 : Data loaded to Database as a table, Executing queries
2025-07-17 00:10:56 : Process Complete
2025-07-17 00:11:08 : Server Connection closed



### Change Log


|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2025-07-16  | 0.1  | Hoang H. Nguyen | Initial version |